## 초기 환경 세팅 및 확인
1. requirements.txt 설치
2. torch는 cuda버전을 사용하고 있으므로 사용자가 추가로 다운로드 필요
3. 그래픽카드 사용 가능여부 확인
    - get_device_name에서 안뜨면 그래픽카드 안잡힌것.
    - 만약 안잡히면 !nvidia-smi 주석을 풀고 그래픽카드가 제대로 잡히는지 확인

In [ ]:
%pip install -r requirements.txt

In [7]:
import torch
import sys
from config import ModelConfig as cf

print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 2070 SUPER


In [10]:
!pip install bitsandbytes==0.43.2

   ---------------------------------------- 0.0/136.5 MB ? eta -:--:--
   ---------------------------------------- 1.0/136.5 MB 49.2 MB/s eta 0:00:03
   - -------------------------------------- 3.7/136.5 MB 8.4 MB/s eta 0:00:16
   ---- ----------------------------------- 14.7/136.5 MB 23.6 MB/s eta 0:00:06
   ----- ---------------------------------- 18.6/136.5 MB 22.1 MB/s eta 0:00:06
   ------ --------------------------------- 21.5/136.5 MB 20.3 MB/s eta 0:00:06
   ------ --------------------------------- 23.1/136.5 MB 18.2 MB/s eta 0:00:07
   ------- -------------------------------- 25.7/136.5 MB 17.3 MB/s eta 0:00:07
   --------- ------------------------------ 33.6/136.5 MB 20.7 MB/s eta 0:00:05
   ----------- ---------------------------- 38.5/136.5 MB 20.2 MB/s eta 0:00:05
   ------------ --------------------------- 41.4/136.5 MB 19.5 MB/s eta 0:00:05
   ------------ --------------------------- 43.5/136.5 MB 18.7 MB/s eta 0:00:05
   ------------- -------------------------- 46.4/136

  You can safely remove it manually.


**torch로 그래픽카드가 잡히지 않을 때만 확인용으로 주석 제거 후 돌리기**

In [ ]:
#!nvidia-smi

## 모델 가져오기
- 현재 파일에서는 QWEN3 0.6B 모델을 사용할 예정입니다.
- 다른 모델을 사용하기 원할 경우 huggingface의 가이드에 따라 모델명을 변경하면 됩니다.

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_8bit=True,   # 8비트로 당겨와서 GPU 메모리 절약할거임
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


### LORA를 통해 모델 상태 확인

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Qwen은 Q/V projection에 적용
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 파라미터가 1~200만 정도 수준이면 잘 된거
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,293,760 || all params: 598,343,680 || trainable%: 0.3833515881708653


## 학습 시킬 데이터 셋. json 형식으로 준비할 것.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/train.json")

def format_example(example):
    return tokenizer(
        f"<|user|>: {example['instruction']}\n<|assistant|>: {example['output']}",
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(format_example)


## 학습 실행

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qwen3-finetuned",     # 여기로 파인튜닝된 모델 저장
    per_device_train_batch_size=2,      
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)
trainer.train()


## LORA 가중치 병합

In [ ]:
model.save_pretrained("./qwen3-lora")
tokenizer.save_pretrained("./qwen3-lora")

from peft import merge_and_unload
merged_model = merge_and_unload(model)
merged_model.save_pretrained("./qwen3-merged")
